In [49]:
import config
import spotipy
import json
import pandas as pd
import os
import pprint
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np
from IPython.display import IFrame 
CLIENT_ID = os.getenv("spotify_client_id")
CLIENT_SECRET = os.getenv("spotify_client_secret_id")

In [50]:
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))

In [51]:
#User gives the name of a song they like

user_song_title = input("Please provide the name of a song so we can recommend similar songs: ")

In [52]:
# Song name sent to spotipy, and spotipy will returns a list of song names

results = sp.search(q=user_song_title, limit=3, market='FR')
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Let+Down&type=track&market=FR&offset=0&limit=3',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
       'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
       'id': '4Z8W4fKeB5YxbusRsdQVPb',
       'name': 'Radiohead',
       'type': 'artist',
       'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6dVIqQ8qmQ5GBnJ9shOYGE'},
     'href': 'https://api.spotify.com/v1/albums/6dVIqQ8qmQ5GBnJ9shOYGE',
     'id': '6dVIqQ8qmQ5GBnJ9shOYGE',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273c8b444df094279e70d0ed856',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02c8b444df094279e70d0ed856',
       'width': 300},
      {'height': 64,
       'url': 'https://i.scd

In [53]:
# From the 3 options returned by Spotipy, extract the song name, artist name, album name, and song ID.

results['tracks']['items'][0]['external_urls'] 
results['tracks']['items'][0]['artists'][0]['name']
results['tracks']['items'][0]['name']
results['tracks']['items'][0]['id']

'2fuYa3Lx06QQJAm0MjztKr'

In [54]:
# Printing the details of the songs returned by Spotipy to the user for their verification.
for i, item in enumerate(results['tracks']['items'], start=1):
    artists = ", ".join(artist['name'] for artist in item['artists'])
    print(f"Song {i}: Song name: {item['name']} | Artist: {artists} | Album: {item['album']['name']}")
    print("")

# User is asked to confirm which of the 3 songs is the song they choose.
user_song_confirmed = input("Which is your song (Song 1/Song 2/Song 3): ").lower()

# Mapping user input to track ID
track_id_mapping = {f"song {i}": item['id'] for i, item in enumerate(results['tracks']['items'], start=1)}

# Check if user input is valid and map it to the corresponding track ID
while user_song_confirmed not in track_id_mapping:
    print("Invalid input. Please enter 'Song 1', 'Song 2', or 'Song 3'.")
    user_song_confirmed = input("Which is your song (Song 1/Song 2/Song 3): ").lower()

# Get the track ID corresponding to the user's input
selected_track_id = track_id_mapping[user_song_confirmed]

# Get the details of the selected song
selected_item = next(item for item in results['tracks']['items'] if item['id'] == selected_track_id)
selected_artists = ", ".join(artist['name'] for artist in selected_item['artists'])

# Print the selected song details
print(f"You selected {user_song_confirmed}: '{selected_item['name']}' by {selected_artists}")

Song 1: Song name: Let Down | Artist: Radiohead | Album: OK Computer

Song 2: Song name: Let Down | Artist: Easy Star All-Stars, Toots & The Maytals | Album: Radiodread

Song 3: Song name: Let Down - Remastered | Artist: Radiohead | Album: OK Computer OKNOTOK 1997 2017

Invalid input. Please enter 'Song 1', 'Song 2', or 'Song 3'.
You selected song 1: 'Let Down' by Radiohead


In [55]:
# Track ID of the user's song sent to Spotipy and that song's audio features are returned. 
selected_track_features = sp.audio_features(selected_track_id)
selected_track_features


[{'danceability': 0.351,
  'energy': 0.676,
  'key': 9,
  'loudness': -9.017,
  'mode': 1,
  'speechiness': 0.0313,
  'acousticness': 0.000121,
  'instrumentalness': 0.113,
  'liveness': 0.18,
  'valence': 0.143,
  'tempo': 102.46,
  'type': 'audio_features',
  'id': '2fuYa3Lx06QQJAm0MjztKr',
  'uri': 'spotify:track:2fuYa3Lx06QQJAm0MjztKr',
  'track_href': 'https://api.spotify.com/v1/tracks/2fuYa3Lx06QQJAm0MjztKr',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2fuYa3Lx06QQJAm0MjztKr',
  'duration_ms': 299560,
  'time_signature': 4}]

In [56]:
# Reshaping selected song audio features

track_feature_dict = selected_track_features[0]

# Extract feature values from the dictionary
feature_values = np.array([
    track_feature_dict['danceability'],
    track_feature_dict['speechiness'],
    track_feature_dict['acousticness'],
])

# Reshape feature_values into a 2D array
feature_values = feature_values.reshape(1, -1)

In [57]:
# Scaling selected song audio features

import pickle

# Load the MinMaxScaler object from the pickle file
with open('scaler6.pickle', 'rb') as handle:
    scaler = pickle.load(handle)

# Scale the feature values
scaled_user_song_features = scaler.transform(feature_values)


c:\Users\darce\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [58]:
# Predicting the cluster of the selected song

# Load the trained KMeans model from the pickle file
with open('model6_km100.pickle', 'rb') as handle:
    kmeans100 = pickle.load(handle)

# Predict the cluster of the selected song
predicted_cluster = kmeans100.predict(scaled_user_song_features)

print("Predicted Cluster:", predicted_cluster)

Predicted Cluster: [21]


In [59]:
clustered_tracks_df_6 = pd.read_csv("tracks_clustered_df_6.csv")
display(clustered_tracks_df_6)

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_href,cluster_km100
0,6YLlHjowYdcrULkRk4r4Ga,0.691,0.519,0,-7.770,1,0.0353,0.686000,0.076700,0.2690,0.11700,125.008,197045,4,https://api.spotify.com/v1/tracks/6YLlHjowYdcr...,79
1,2F2p7b5Xq20mRyEeWYaeUF,0.690,0.890,0,-4.866,0,0.0507,0.009030,0.000016,0.1880,0.71300,124.018,181478,4,https://api.spotify.com/v1/tracks/2F2p7b5Xq20m...,55
2,17B9XlyyYDKkTmPay4z02V,0.587,0.722,2,-6.983,0,0.0709,0.000754,0.668000,0.5430,0.46600,121.964,180843,4,https://api.spotify.com/v1/tracks/17B9XlyyYDKk...,12
3,1cYyZ1N98GyRS3cMcccFHd,0.734,0.855,6,-4.578,0,0.1350,0.082100,0.000005,0.0901,0.54000,121.955,139563,4,https://api.spotify.com/v1/tracks/1cYyZ1N98GyR...,47
4,2a1o6ZejUi8U3wzzOtCOYw,0.400,0.915,0,-4.890,1,0.2800,0.084600,0.000000,0.0503,0.87800,88.917,198773,3,https://api.spotify.com/v1/tracks/2a1o6ZejUi8U...,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36260,77wRBCrqhWvvQjEI44uHEi,0.182,0.942,10,-22.021,0,0.0621,0.000016,0.965000,0.3730,0.02790,80.479,113961,3,https://api.spotify.com/v1/tracks/77wRBCrqhWvv...,9
36261,3imggdpnHlcy8I25zT93KW,0.103,0.374,1,-22.173,1,0.0459,0.000000,0.993000,0.4650,0.07070,91.115,156511,3,https://api.spotify.com/v1/tracks/3imggdpnHlcy...,9
36262,7kMw28F523wOc3OLveJSF2,0.172,1.000,9,-23.813,0,0.0653,0.695000,0.556000,0.8340,0.00001,117.909,125631,1,https://api.spotify.com/v1/tracks/7kMw28F523wO...,75
36263,3agFKTnUIFMbLCtBrnxlqA,0.112,0.994,5,-20.663,0,0.0618,0.134000,0.990000,0.4460,0.01410,47.173,73561,3,https://api.spotify.com/v1/tracks/3agFKTnUIFMb...,80


In [60]:
tracks_in_predicted_cluster = clustered_tracks_df_6[clustered_tracks_df_6['cluster_km100'] == predicted_cluster[0]]
tracks_in_predicted_cluster
import random
random_track = tracks_in_predicted_cluster.sample(n=1)
print("Randomly selected track from the predicted cluster:")
print(random_track)


Randomly selected track from the predicted cluster:
                     track_id  danceability  energy  key  loudness  mode  \
22443  5cMB5jGfNCm6b65YXhSIEJ         0.359   0.899    1    -3.293     0   

       speechiness  acousticness  instrumentalness  liveness  valence  \
22443       0.0592       0.00441               0.0     0.232    0.342   

         tempo  duration_ms  time_signature  \
22443  162.944       167853               4   

                                              track_href  cluster_km100  
22443  https://api.spotify.com/v1/tracks/5cMB5jGfNCm6...             21  
